In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ast
import glob2
import PIL
from tqdm import tqdm

import numpy as np
import shutil
from pathlib import Path

try:
    import Image
except ImportError:
    from PIL import Image

In [2]:
allfiles = glob2.glob('./fivehund_annotations/*.txt')
allfiles[0]

'./fivehund_annotations/gt_img_1.txt'

In [5]:
allfiles = glob2.glob('./fivehund_annotations/*.txt')  #create a list of all txt files

x1 = []  #create an empty list
y1 = []  #create an empty list
x2 = []  #create an empty list
y2 = []  #create an empty list
x3 = []  #create an empty list
y3 = []  #create an empty list
x4 = []  #create an empty list
y4 = []  #create an empty list
labels = []  #create an empty list
names = []  #create an empty list
X1 = []  #create an empty list
Y1 = []  #create an empty list
X2 = []  #create an empty list
Y2 = []  #create an empty list
image_width = []  #create an empty list
image_height = []  #create an empty list


for txt in tqdm(allfiles):  #iterate over each txt file
    
    with open(txt, 'r', encoding='utf-8-sig') as f:  #read through a txt file
        lines = f.readlines()  #store the read lines into a list
        
        gt_name = txt.split('/')[2].split('.')[0]  #get this part 'gt_img_1'
        name = gt_name.removeprefix('gt_')  #get this part 'img_1'

        #print(len(lines))


        for thing in lines:    #iterate over each line
            
            x_up_left = int(thing.split(',')[0])  #store the corresponding coordinate
            y_up_left = int(thing.split(',')[1])  #store the corresponding coordinate
            x_up_right = int(thing.split(',')[2]) #store the corresponding coordinate
            y_up_right = int(thing.split(',')[3])  #store the corresponding coordinate
            x_down_right = int(thing.split(',')[4])  #store the corresponding coordinate
            y_down_right = int(thing.split(',')[5])  #store the corresponding coordinate
            x_down_left = int(thing.split(',')[6])  #store the corresponding coordinate
            y_down_left = int(thing.split(',')[7])  #store the corresponding coordinate
            
            X_up_left = min(x_up_left, x_down_left)  #take the min for the rectangle
            Y_up_left = min(y_up_left, y_up_right)  #take the min for the rectangle
            X_down_right = max(x_up_right, x_down_right)  #take the max for the rectangle
            Y_down_right = max(y_down_right, y_down_left)  #take the max for the rectangle
            
            image = PIL.Image.open(f'./fivehund_images/{name}.jpg')  #read coresponding images
            width, height = image.size  #store the dimensions
            
            image_width.append(image.size[0])  #add the width to the image_width list
            image_height.append(image.size[1])  #add the height to the image_height list
            
            
            x1.append(int(x_up_left))  #add to the x1 list
            y1.append(int(y_up_left))  #add to the y1 list
            x2.append(int(x_up_right))  #add to the x2 list
            y2.append(int(y_up_right))  #add to the y2 list
            x3.append(int(x_down_right))  #add to the x3 list
            y3.append(int(y_down_right))  #add to the y3 list
            x4.append(int(x_down_left))  #add to the x4 list
            y4.append(int(y_down_left))  #add to the y4 list
            
            X1.append(int(X_up_left))  #add to the X1 list
            Y1.append(int(Y_up_left))  #add to the Y1 list
            X2.append(int(X_down_right))  #add to the X2 list
            Y2.append(int(Y_down_right))  #add to the Y2 list
            
            labels.append(thing.split(',')[8].replace("\n", ""))  #add to the labes list
            names.append(name)

        #x1 = list(map(int, x1))  #transform to a list of int
        #y1 = list(map(int, y1))  #transform to a list of int
        #x2 = list(map(int, x2))  #transform to a list of int
        #y2 = list(map(int, y2))  #transform to a list of int
        #x3 = list(map(int, x3))  #transform to a list of int
        #y3 = list(map(int, y3))  #transform to a list of int
        #x4 = list(map(int, x4))  #transform to a list of int
        #y4 = list(map(int, y4))  #transform to a list of int
        
        
        
#create a dataframe containig the needed lists
frame = pd.DataFrame(list(zip(names, labels, X1, Y1, X2, Y2, image_width, image_height)), 
               columns =['imageName', 'bbox', 'X1', 'Y1', 'X2', 'Y2', 'image_width', 'image_height']) 
frame

100%|██████████| 500/500 [00:03<00:00, 165.37it/s]


,imageName,bbox,X1,Y1,X2,Y2,image_width,image_height
0,img_1,###,933,255,956,277,1280,720
1,img_1,###,172,323,195,339,1280,720
2,img_1,###,83,270,118,294,1280,720
3,img_1,###,940,310,962,320,1280,720
4,img_1,###,946,351,978,374,1280,720
...,...,...,...,...,...,...,...,...
5225,img_499,TURNING,347,236,395,261,1280,720
5226,img_499,INTO,397,231,425,252,1280,720
5227,img_499,ISTANA,342,256,382,280,1280,720
5228,img_499,DOMAIN,384,246,427,272,1280,720


In [6]:
frame.to_csv(r'fivehund.csv', index = False) #save to csv

In [7]:
unik = frame['imageName'].unique().tolist()

for noms in tqdm(unik):
    #firstimage = (dataset['imageName'] == noms)
    groupe_of_first_image = frame[frame['imageName'] == noms]
    groupe = groupe_of_first_image.reset_index(drop=True)
    
    matrix = (np.zeros(((groupe.iloc[0, 7]), (groupe.iloc[0, 6]))))
    for i in range(len(groupe)):

        #matrix = (np.zeros(((groupe.iloc[i, 9]), (groupe.iloc[i, 8]))))

        block = (np.zeros(((groupe.iloc[i, 7]), (groupe.iloc[i, 6]))))

        #block = (np.ones(((groupe.iloc[i, 6]) - (groupe.iloc[i, 4])), ((groupe.iloc[i, 5]) - (groupe.iloc[i, 3]))))

        block[(groupe.iloc[i, 3]):(groupe.iloc[i, 5]), (groupe.iloc[i, 2]):(groupe.iloc[i, 4])] = 1

        matrix = np.add(matrix, block) #matrix + block

    #plt.imshow(matrix)
    #plt.show()
    img = Image.fromarray(matrix)
    img = img.convert('RGB')
    img.save(f'./fivehund_masks/{noms}.jpg')

100%|██████████| 500/500 [01:21<00:00,  6.12it/s]


In [10]:
unik = frame['imageName'].unique().tolist()

for noms in tqdm(unik):

    groupe_of_first_image = frame[frame['imageName'] == noms]  #enumerate only the first image
    groupe = groupe_of_first_image.reset_index(drop=True)  #drop index


    gloc = (np.zeros((4, (groupe.iloc[0, 7]), (groupe.iloc[0, 6])))) #the big tensor of dim(4,m,n)
    flop = (np.zeros(((groupe.iloc[0, 7]), (groupe.iloc[0, 6]))))  #large matrix of dim(m,n)
    width = (groupe.iloc[0, 6])  #store the width
    height = (groupe.iloc[0, 7])  #store the height
    x_indices = np.indices((height, width))[0] #matrix of x indices
    y_indices = np.indices((height, width))[1] #matrix of y indices


    for i in range(len(groupe)):  #iterate over the length of the 'group'



        block = (np.zeros(((groupe.iloc[i, 7]), (groupe.iloc[i, 6]))))   #large matrix of dim(m,n)



        block[(groupe.iloc[i, 3]):(groupe.iloc[i, 5]), (groupe.iloc[i, 2]):(groupe.iloc[i, 4])] = 1  #fill the block w/ 1s

        flop = np.add(flop, block) #matrix + block

        x_up_left = (groupe.iloc[i, 2])  #x of upleft corner of bbox
        y_up_left = (groupe.iloc[i, 3])  #y of upleft corner of bbox



        x_up_right = (groupe.iloc[i, 4])  #x of upright corner of bbox
        y_up_right = (groupe.iloc[i, 3])  #y of upright corner of bbox



        x_down_right = (groupe.iloc[i, 4])  #x of downright corner of bbox 
        y_down_right = (groupe.iloc[i, 5])  #y of downright corner of bbox 

        x_down_left = (groupe.iloc[i, 2])  #x of downleft corner of bbox 
        y_down_left = (groupe.iloc[i, 5])  #y of downleft corner of bbox 



        x_indices_mask = np.multiply(flop, x_indices)  #matrix with masks for x indices 
        y_indices_mask = np.multiply(flop, y_indices)  #matrix with masks for y indices 






        distance_from_up_left = (x_indices_mask - x_up_left)**2 + (y_indices_mask - y_up_left)**2  #compute the distance
        distance_from_up_left = np.multiply(distance_from_up_left,flop) #hadamard product with the masks
        distance_from_up_left = np.abs(distance_from_up_left) #absolute values 
        distance_from_up_left = np.sqrt(distance_from_up_left) #square root

        gloc[0] = distance_from_up_left  #fill in the first dim of the big tensor 

        distance_from_up_right = (x_indices_mask - x_up_right)**2 + (y_indices_mask - y_up_right)**2  #compute the distance
        distance_from_up_right = np.multiply(distance_from_up_right,flop) #hadamard product with the masks
        distance_from_up_right = np.abs(distance_from_up_right) #absolute values 
        distance_from_up_right = np.sqrt(distance_from_up_right) #square root

        gloc[1] = distance_from_up_right  #fill in the second dim of the big tensor

        distance_from_down_right = (x_indices_mask - x_down_right)**2 + (y_indices_mask - y_down_right)**2 #compute the distance
        distance_from_down_right = np.multiply(distance_from_down_right,flop) #hadamard product with the masks
        distance_from_down_right = np.abs(distance_from_down_right) #absolute values 
        distance_from_down_right = np.sqrt(distance_from_down_right) #square root

        gloc[2] = distance_from_down_right  #fill in the third dim of the big tensor

        distance_from_down_left = (x_indices_mask - x_down_left)**2 + (y_indices_mask - y_down_left)**2 #compute the distance
        distance_from_down_left = np.multiply(distance_from_down_left,flop) #hadamard product with the masks
        distance_from_down_left = np.abs(distance_from_down_left) #absolute values 
        distance_from_down_left = np.sqrt(distance_from_down_left) #square root

        gloc[3] = distance_from_down_left  #fill in the fourth dim of the big tensor


    np.save(f'./fivehund_distancegeo/{noms}.npy',gloc)



    #plotting    
    #for w in range(4):

    #    plt.imshow(gloc[w])
    #    plt.show()

100%|██████████| 500/500 [16:27<00:00,  1.98s/it]
